# Task 14 - Parameter Study Optimisation

The previous task sampled from the available parameters and aimed to cover the parameter space efficiently. This task uses Scikit-opt Gaussian Processing to home-in on the optimal solution. In optimisation algorithms it is common to see a combination of exploration and exploitation to find the optimal value.

In [ ]:
# Install dependencies

import json
import numpy as np
import pandas as pd
import adaptive
import holoviews
import ipywidgets
import nest_asyncio
import plotly.graph_objects as go

from tqdm import tqdm
from pathlib import Path
from skopt import gp_minimize
from skopt.utils import dump, load
from scipy.interpolate import griddata

from openmc_model import objective

adaptive.notebook_extension()
nest_asyncio.apply()

# method for saving results in json file
def output_result(filepath, result):
    filename = filepath
    Path(filename).parent.mkdir(parents=True, exist_ok=True)
    with open(filename, mode="w", encoding="utf-8") as f:
        json.dump(result, f, indent=4)

So that we have data to compare our optimisation values to, first we will perform a sweep to see how TBR varies across the parameter space. In these simulations, we will use a pre-defined model to see how TBR varies as a function of breeder_percent_in_breeder_plus_multiplier.

Run the code below to perform this 1D parameter sweep.

In [ ]:
# get_true_values_1D

tbr_values = []
for breeder_percent_in_breeder_plus_multiplier in tqdm(np.linspace(0, 100, 101)):
    tbr_values.append({'breeder_percent_in_breeder_plus_multiplier':breeder_percent_in_breeder_plus_multiplier,
                       'tbr':-objective([breeder_percent_in_breeder_plus_multiplier])})
    
output_result("outputs/1d_tbr_values.json", tbr_values)

The code block below runs an optimised simualation. First, the simulation takes some sample points of the parameter space and fits the data using gaussian processing. It will then run a simulation at the point where TBR is maximum and fit again. It does this iteratively for a set number of times - should get close to the maximum TBR across that parameter space.
Enough simulations must be run to get to the correct point in the parameter space.
This is using the same adaptive sampling techniques as task 13.

In [ ]:
# get_optimised_values_1d

learner = adaptive.SKOptLearner(objective,
                                dimensions=[(0., 100.)],
                                base_estimator="GP",
                                acq_func="gp_hedge",
                                acq_optimizer="lbfgs",
                               )
runner = adaptive.Runner(learner, ntasks=1, goal=lambda l: l.npoints > 40)

runner.live_info()

runner.ioloop.run_until_complete(runner.task)

output_result("outputs/1d_optimised_values.json", dict(learner.data))

We can then plot the true data (from the parameter sweep) and the optimisation data to see whether in fact the optimisation has reached the correct point where TBR is maximum.

In [ ]:
# 1_plot_1d_optimisation
# Note, optimisation functions tend to minimise the value therefore there are a few negative signs in these scripts

# Loads true data for comparison
data = pd.read_json('outputs/1d_tbr_values.json')

x_data=data['breeder_percent_in_breeder_plus_multiplier']
fx=-data['tbr']


# Load optimisation data

with open('outputs/1d_optimised_values.json', 'r') as f: data = json.load(f).items()

x_vals = [i[0] for i in data]
tbr_vals = [-i[1] for i in data]

# Print max TBR from optimisation data
print('Maximum TBR of ', tbr_vals[-1], 'found with a breeder percent in breeder plus multiplier of ', x_vals[-1])


fig = go.Figure()

# Plot samples from optimsation points
fig.add_trace(go.Scatter(x = x_vals,
                         y = tbr_vals,
                         name="Samples from optimisation",
                         mode='markers',
                         marker=dict(color='red', size=10)
                        )
                )

# Plot true function.
fig.add_trace(go.Scatter(name="True value (unknown)",
                          x = x_data,
                          y = [-i for i in fx],
                          mode='lines',
                          line = {'shape': 'spline'},
                          marker=dict(color='green')
                         )
              )

fig.update_layout(title='Optimal breeder percent in breeder plus multiplier',
                  xaxis={'title': 'breeder percent in breeder plus multiplier', 'range': [0, 100]},
                  yaxis={'title': 'TBR', 'range': [0.1, 2]}
                 )

fig.show()

As you can see, the optimisation samples are spread across the parameter space, but get more dense towards a single point. This is because the optimisation is homing-in on the parameter space where TBR is maximum so more samples will be taken here until we get very close to the max TBR.

The same techniques can be applied to N dimensional problems but the number of simulations required increases. The next example is a 2-dimensional problem where the optimal breeder to multiplier ratio and enrichment are being found.

First, a sweep of the parameter space is performed to obtain TBR as a function of breeder to multiplier ratio and enrichment across the entire parameter space.

In [ ]:
# get_true_values_2D

tbr_values = []
for breeder_percent_in_breeder_plus_multiplier in tqdm(np.linspace(0, 100, 10)):
    for blanket_breeder_li6_enrichment in np.linspace(0, 100, 10):
        tbr_values.append({'breeder_percent_in_breeder_plus_multiplier': breeder_percent_in_breeder_plus_multiplier,
                           'blanket_breeder_li6_enrichment': blanket_breeder_li6_enrichment,
                           'tbr': -objective([breeder_percent_in_breeder_plus_multiplier,
                                              blanket_breeder_li6_enrichment])
                          })

output_result("outputs/2d_tbr_values.json", tbr_values)

Then we run an optimised simulation. First we use adaptive sampling like before, but this time in 2D, and then we can also use gp_optimisation in 2D?
Not really too sure why two optimisations seemed to be used here.

In [ ]:
# get_optimised_values_2d

# Uses adaptive sampling methods from task X to obtain starting points for the optimiser
learner = adaptive.Learner2D(objective, bounds=[(0, 100), (0, 100)])
runner = adaptive.Runner(learner, ntasks=1, goal=lambda l: l.npoints > 40)
runner.live_info()
runner.ioloop.run_until_complete(runner.task)


# Gaussian Processes based optimisation that returns an SciPy optimisation object
res = gp_minimize(objective,          # the function to minimize
                  dimensions=[(0., 100.), (0., 100.)],       # the bounds on each dimension of x
                  n_calls=40,         # the number of evaluations of f
                  n_random_starts=0,  # the number of random initialization points
                  verbose=True,
                  x0=[i for i in list(learner.data.keys())], # initial data from the adaptive sampling method
                  y0=list(learner.data.values()) # initial data from the adaptive sampling method
                  )

# saves 2d optimisation results in .dat file
dump(res, "outputs/2d_optimised_values.dat")

Plot the true and optimised data on the same graph to show whether the optimisation data got close to the true max TBR across the parameter space.

In [ ]:
# 2d_plot_2d_optimisation_scatter.py

# load true data for comparison
data = pd.read_json('outputs/2d_tbr_values.json')
x=data['breeder_percent_in_breeder_plus_multiplier']
y=data['blanket_breeder_li6_enrichment']
z=data['tbr']

# Print max TBR from optimisation data
print('Optimal breeder_percent_in_breeder_plus_multiplier_ratio = ', res.x[0])
print('Optimal Li6 enrichment = ', res.x[1])
print('Maximum TBR = ', -res.fun)

fig = go.Figure()

fig.add_trace(go.Scatter3d(name='TBR values found during optimisation',
                           x=[x[0] for x in res.x_iters],
                           y=[x[1] for x in res.x_iters],
                           z=-res.func_vals,
                           mode='markers',
                           marker=dict(size=7)
                          )
             )

fig.add_trace(go.Scatter3d(name='True values',
                           x=x,
                           y=y,
                           z=z,
                           mode='markers',
                           marker=dict(size=7)
                          )
             )


fig.add_trace(go.Scatter3d(name='Maximum TBR value found',
                           x=[res.x[0]],
                           y=[res.x[1]],
                           z=[-res.fun],
                           mode='markers',
                           marker=dict(size=7)
                          )
             )

fig.update_layout(title='Optimal Li6 enrichment and breeder percent in breeder plus multiplier',
                  scene={'yaxis': {'title': 'Li6 enrichment percent'},
                         'zaxis': {'title': 'breeder percent in breeder plus multiplier'},
                         'zaxis': {'title': 'TBR'}
                        }
                 )

fig.show()

Can also produce a contour plot to show a similar comparison.

In [ ]:
# 2_plot_2d_optimisation_contour

# Print max TBR from optimisation data
print('Optimal Li6 enrichment = ', res.x[0])
print('Optimal breeder percent in breeder plus multiplier = ', res.x[1])
print('Maximum TBR = ', -res.fun)


# creates a grid and interploates values on it
xi = np.linspace(0, 100, 100)
yi = np.linspace(0, 100, 100)
zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='linear')


fig = go.Figure()

# plots interpolated values as colour map plot
fig.add_trace(trace = go.Contour(
                z=zi,
                x=yi,
                y=xi,
        colorscale="Viridis",
        opacity=0.9,
        line=dict(width=0, smoothing=0.85),
        contours=dict(
            showlines=False,
            showlabels=False,
            size=0,
            labelfont=dict(size=15,),
        ),
    ))

fig.add_trace(go.Scatter(name='TBR values found during optimisation',
                         x=[x[0] for x in res.x_iters],
                         y=[x[1] for x in res.x_iters],
                         hovertext=-res.func_vals,
                         hoverinfo="text",
                         marker={"size": 8},
                         mode='markers'
                        )
             )


# This add the final optimal value found during the optimisation as a seperate scatter point on the graph
fig.add_trace(go.Scatter(name='Maximum TBR value found',
                         x=[res.x[0]],
                         y=[res.x[1]],
                         hovertext=[-res.fun],
                         hoverinfo="text",
                         marker={"size": 8},
                         mode='markers'
                        )
             )

fig.update_layout(title='',
                  xaxis={'title': 'breeder percent in breeder plus multiplier', 'range':(-1, 101)},
                  yaxis={'title': 'blanket breeder li6 enrichment', 'range':(-1, 101)},
                  legend_orientation="h"
                 )

fig.show()

As you can see, the optimisation points are spread across the parameter space but are more dense in towards the area of maximum TBR (as given by the real data).